Necessary imports

In [21]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import RandomLayers
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

Setting the hyperparameters of the model

In [22]:
n_epochs=10
n_layers=1

In [23]:
SAVE_PATH="/home/user/Desktop/implementation/"
PREPROCESS=True
np.random.seed(0)
tf.random.set_seed(0)

loading the data

In [24]:
mnist_dataset=keras.datasets.mnist
(train_images,train_labels),(test_images,test_labels)=mnist_dataset.load_data()
test_images.shape

(10000, 28, 28)

normalising the data

In [25]:
train_images=train_images/255
test_images=test_images/255
train_images.shape

(60000, 28, 28)

adding extra dimension for convolution channels

In [26]:
train_images=np.array(train_images[...,tf.newaxis],requires_grad=False)
test_images=np.array(test_images[...,tf.newaxis],requires_grad=False)#requires_grad??
train_images.shape

(60000, 28, 28, 1)

initilaizing a default.qubit device simulating a system of 4 qubits

In [27]:
dev = qml.device("default.qubit", wires=4)

Initialising random parameters

In [28]:
rand_params = np.random.uniform(high=2 * np.pi, size=(n_layers, 4))#1 x 4 returned as tensor
rand_params

tensor([[3.44829694, 4.49366732, 3.78727399, 3.42360201]], requires_grad=True)

qnode consists of a quantum function and a device on which it executes

In [29]:
@qml.qnode(dev)
def circuit(phi):
    # Encoding of 4 classical input values
    for j in range(4):
        qml.RY(np.pi * phi[j], wires=j)#an embedding layer of local Ry rotations

    # Random quantum circuit
    RandomLayers(rand_params, wires=list(range(4)))#a random circuit of n_layers rand_params is the weight first argument of weight is the number of layers and the second argument of weights is the number of rotations

    # Measurement producing 4 classical output values
    return [qml.expval(qml.PauliZ(j)) for j in range(4)]#measurement in the computational basis

drawing the circuit for illustration purposes with random parameters

In [30]:
drawer = qml.draw(circuit)
print(drawer([1,2,3,4]))

0: ──RY(3.14)──╭RandomLayers(M0)─┤  <Z>
1: ──RY(6.28)──├RandomLayers(M0)─┤  <Z>
2: ──RY(9.42)──├RandomLayers(M0)─┤  <Z>
3: ──RY(12.57)─╰RandomLayers(M0)─┤  <Z>


Convolution the input image with many applications of the same quantum circuit.

In [31]:
def quanv(image):
    out = np.zeros((14, 14, 4))#initilaising output image with 4 different channels

    # Loop over the coordinates of the top-left pixel of 2X2 squares
    for j in range(0, 28, 2):
        for k in range(0, 28, 2):
            # Process a squared 2x2 region of the image with a quantum circuit
            q_results = circuit(
                [
                    image[j, k, 0],
                    image[j, k + 1, 0],
                    image[j + 1, k, 0],
                    image[j + 1, k + 1, 0]
                ]
            )
            # Assign expectation values to different channels of the output pixel (j/2, k/2)
            for c in range(4):#the 4 expectation values are mapped into 4 different channels of a single output pixel
                out[j // 2, k // 2, c] = q_results[c]
    return out


preprocessing the train and test images as the quanvoluional layer isnt trained

In [32]:
if PREPROCESS == True:
    q_train_images = []
    print("Quantum pre-processing of train images:")
    for idx, img in enumerate(train_images):
        print("{}/{}        ".format(idx + 1, 60000), end="\r")
        q_train_images.append(quanv(img))
    q_train_images = np.asarray(q_train_images)

    q_test_images = []
    print("\nQuantum pre-processing of test images:")
    for idx, img in enumerate(test_images):
        print("{}/{}        ".format(idx + 1,10000 ), end="\r")
        q_test_images.append(quanv(img))
    q_test_images = np.asarray(q_test_images)

    # Save pre-processed images
    np.save(SAVE_PATH + "q_train_images.npy", q_train_images)
    np.save(SAVE_PATH + "q_test_images.npy", q_test_images)

    print(q_train_images.shape)

# Load pre-processed images
q_train_images = np.load(SAVE_PATH + "q_train_images.npy")
q_test_images = np.load(SAVE_PATH + "q_test_images.npy")

Quantum pre-processing of train images:
60000/60000        
Quantum pre-processing of test images:
(60000, 14, 14, 4) 


In [43]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
def MyModel():
    """Initializes and returns a custom Keras model
    which is ready to be trained."""
    model=Sequential()
    model.add(Conv2D(32,(3,3),activation='relu',input_shape=(14,14,4)))#convolution layer
    model.add(MaxPool2D(2,2))#pooling layer

    model.add(Flatten())#flattening to form 1 D array
    model.add(Dense(100,activation='relu'))# hidden layer
    model.add(Dense(10,activation='softmax'))#output layer

    model.compile(
        optimizer='adam',
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model
q_model = MyModel()

q_history = q_model.fit(
    q_train_images,
    train_labels,
    batch_size=4,
    epochs=n_epochs,
    verbose=2,
)

Epoch 1/10
15000/15000 - 27s - loss: 0.1232 - accuracy: 0.9626 - 27s/epoch - 2ms/step
Epoch 2/10
15000/15000 - 27s - loss: 0.0555 - accuracy: 0.9827 - 27s/epoch - 2ms/step
Epoch 3/10
15000/15000 - 26s - loss: 0.0389 - accuracy: 0.9881 - 26s/epoch - 2ms/step
Epoch 4/10
15000/15000 - 26s - loss: 0.0345 - accuracy: 0.9895 - 26s/epoch - 2ms/step
Epoch 5/10
15000/15000 - 27s - loss: 0.0256 - accuracy: 0.9926 - 27s/epoch - 2ms/step
Epoch 6/10
15000/15000 - 27s - loss: 0.0230 - accuracy: 0.9936 - 27s/epoch - 2ms/step
Epoch 7/10
15000/15000 - 26s - loss: 0.0213 - accuracy: 0.9938 - 26s/epoch - 2ms/step
Epoch 8/10
15000/15000 - 26s - loss: 0.0202 - accuracy: 0.9948 - 26s/epoch - 2ms/step
Epoch 9/10
15000/15000 - 27s - loss: 0.0195 - accuracy: 0.9952 - 27s/epoch - 2ms/step
Epoch 10/10
15000/15000 - 27s - loss: 0.0156 - accuracy: 0.9959 - 27s/epoch - 2ms/step


Evaluation of model

In [44]:
q_model.evaluate(q_test_images,test_labels)

313/313 [==============================] - 1s 2ms/step - loss: 0.1051 - accuracy: 0.9857


[0.10511688143014908, 0.9857000112533569]